In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
from shapely import wkt
import fiona

https://hub.arcgis.com/datasets/6996f03a1b364dbab4008d99380370ed_0/explore?location=0.969507%2C1.535162%2C1.73

In [2]:
world = gpd.read_file("World_Cities.geojson")

In [3]:
world['capital'] = world['STATUS'].apply(lambda x: 1 if 'national' in x.lower() else 0)

In [4]:
world.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
layers = fiona.listlayers("World_Water_Bodies/v107/hydropolys.gdb")

In [6]:
layers

['hydropolys']

https://hub.arcgis.com/content/e750071279bf450cbd510454a80f2e63/about

In [7]:
water = gpd.read_file("World_Water_Bodies/v107/hydropolys.gdb", layer="hydropolys")

In [8]:
water.head()

,Name1,Name2,Name3,TYPE,ISO_CC,SHAPE_Length,SHAPE_Area,geometry
0,,,,Inland intermittent,,0.084017,0.000161,"MULTIPOLYGON (((33.36209 -3.57682, 33.36398 -3..."
1,,,,Inland intermittent,,0.122991,0.000206,"MULTIPOLYGON (((13.74117 5.63882, 13.74251 5.6..."
2,,,,Inland intermittent,,0.039491,0.000059,"MULTIPOLYGON (((35.91590 13.69591, 35.91485 13..."
3,,,,Inland intermittent,,0.109760,0.000323,"MULTIPOLYGON (((-9.91448 15.94434, -9.91341 15..."
4,,,,Inland intermittent,AE,0.014541,0.000012,"MULTIPOLYGON (((53.83962 23.96135, 53.83979 23..."


In [9]:
water.TYPE.unique()

array(['Inland intermittent', 'Dry salt flat', 'Inland perennial',
       'Inundated area', 'Ocean or Sea'], dtype=object)

In [11]:
len(water)

2804476

In [10]:
# Reproject to a projected CRS for distance calculations
projected_crs = 'EPSG:3857'  # Web Mercator projection
world = world.to_crs(projected_crs)
water = water.to_crs(projected_crs)

In [15]:
# Perform the nearest spatial join: https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin_nearest.html
nearest = gpd.sjoin_nearest(world, water, how='inner', lsuffix='_city', rsuffix='_water', distance_col='distance')

In [21]:
nearest.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2540 entries, 0 to 2539
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   FID           2540 non-null   int64   
 1   OBJECTID      2540 non-null   int64   
 2   CITY_NAME     2540 non-null   object  
 3   GMI_ADMIN     2540 non-null   object  
 4   ADMIN_NAME    2540 non-null   object  
 5   FIPS_CNTRY    2540 non-null   object  
 6   CNTRY_NAME    2540 non-null   object  
 7   STATUS        2540 non-null   object  
 8   POP           2540 non-null   int64   
 9   POP_RANK      2540 non-null   int64   
 10  POP_CLASS     2540 non-null   object  
 11  PORT_ID       2540 non-null   int64   
 12  LABEL_FLAG    2540 non-null   int64   
 13  POP_SOURCE    2540 non-null   object  
 14  geometry      2540 non-null   geometry
 15  capital       2540 non-null   int64   
 16  index__water  2540 non-null   int64   
 17  Name1         2540 non-null   object  
 18  Name2

In [22]:
nearest

,FID,OBJECTID,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,...,capital,index__water,Name1,Name2,Name3,TYPE,ISO_CC,SHAPE_Length,SHAPE_Area,distance
0,1,1,Cuiaba,BRA-MGR,Mato Grosso,BR,Brazil,Provincial capital,540814,3,...,0,58201,,,,Inland perennial,BR,5.444046,0.002805,0.903914
1,2,2,Brasilia,BRA-DFD,Distrito Federal,BR,Brazil,National and provincial capital,2481272,2,...,1,2218993,Lago do Paranoá,,,Inland perennial,BR,0.811637,0.002991,4.012811
2,3,3,Goiania,BRA-GOI,Goias,BR,Brazil,Provincial capital,1297154,2,...,0,57486,,,,Inland perennial,BR,0.028151,0.000023,15.550003
3,4,4,Campo Grande,BRA-MGD,Mato Grosso do Sul,BR,Brazil,Provincial capital,776242,3,...,0,54367,,,,Inland perennial,BR,0.012474,0.000008,17.118758
4,5,5,Pedro Juan Caballero,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,...,0,53738,,,,Inland perennial,BR,0.012653,0.000008,14.526046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2536,2536,Greymouth,NZL-WST,Westland,NZ,New Zealand,Provincial capital,0,0,...,0,2448637,Tasman Sea,Pacific Ocean,,Ocean or Sea,,1.058717,0.057883,2.525252
2536,2537,2537,Christchurch,NZL-CNT,Canterbury,NZ,New Zealand,Provincial capital,388500,4,...,0,1337742,,,,Inland perennial,NZ,0.108650,0.000227,6.979020
2537,2538,2538,Dunedin,NZL-OTG,Otago,NZ,New Zealand,Provincial capital,130700,5,...,0,2485728,Pacific Ocean,,,Ocean or Sea,,0.097013,0.000236,3.592056
2538,2539,2539,Invercargill,NZL-STH,Southland,NZ,New Zealand,Provincial capital,0,0,...,0,2600085,Tasman Sea,Pacific Ocean,,Ocean or Sea,,1.497316,0.038836,3.846671


In [18]:
# Convert distances from meters to kilometers
nearest['distance'] = nearest['distance'] / 1000  # Convert to kilometers

In [23]:
df = nearest.drop(columns=['OBJECTID','PORT_ID','LABEL_FLAG'])

In [50]:
# Remove cities without population data and is not the capital
df = df[~((df.capital==0)&(df.POP_RANK==0))] 

In [51]:
# Define a mapping for POP_RANK to radius thresholds
pop_rank_to_radius = {
    1: 40, # 5,000,000 and greater
    2: 30, # 1,000,000 to 4,999,999
    3: 20, # 500,000 to 999,999
    4: 10, # 250,000 to 499,999
    5: 8, # 100,000 to 249,999
    6: 5, # 50,000 to 99,999
    7: 3, # Less than 50,000
    0: 5 #  no population data # row removed so the number does not matter
}

In [52]:
# Function to determine 'type' based on POP_RANK and distance
def determine_type(row):
    pop_rank = row['POP_RANK']
    distance = row['distance']
    
    # Get the corresponding radius for the city's pop_rank
    threshold = pop_rank_to_radius.get(pop_rank)
    
    # Check if distance to coast is less than the threshold
    if distance < threshold:
        return 'water'
    else:
        return 'landlocked'

In [53]:
# Apply the function to create the 'type' column
df['city_type'] = df.apply(determine_type, axis=1)

In [59]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1215 entries, 0 to 2539
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   FID           1215 non-null   int64   
 1   CITY_NAME     1215 non-null   object  
 2   GMI_ADMIN     1215 non-null   object  
 3   ADMIN_NAME    1215 non-null   object  
 4   FIPS_CNTRY    1215 non-null   object  
 5   CNTRY_NAME    1215 non-null   object  
 6   STATUS        1215 non-null   object  
 7   POP           1215 non-null   int64   
 8   POP_RANK      1215 non-null   int64   
 9   POP_CLASS     1215 non-null   object  
 10  POP_SOURCE    1215 non-null   object  
 11  geometry      1215 non-null   geometry
 12  capital       1215 non-null   int64   
 13  index__water  1215 non-null   int64   
 14  Name1         1215 non-null   object  
 15  Name2         1215 non-null   object  
 16  Name3         1215 non-null   object  
 17  TYPE          1215 non-null   object  
 18  ISO_C

In [57]:
# df[df['FIPS_CNTRY']=="CA"] # Canada is all water
df[df['FIPS_CNTRY']=="US"] 

,FID,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,...,index__water,Name1,Name2,Name3,TYPE,ISO_CC,SHAPE_Length,SHAPE_Area,distance,city_type
171,172,Galveston,USA-TEX,Texas,US,United States,Other,50446,6,"50,000 to 99,999",...,2622705,Gulf of Mexico,Atlantic Ocean,,Ocean or Sea,,0.723591,0.006020,0.000000,water
173,174,Juneau,USA-ALK,Alaska,US,United States,Provincial capital,31974,7,"Less than 50,000",...,2711512,Pacific Ocean,,,Ocean or Sea,,1.008181,0.003910,1.023836,water
174,175,Jackson,USA-MSP,Mississippi,US,United States,Provincial capital,4642,7,"Less than 50,000",...,2049665,,,,Inland perennial,US,0.072429,0.000147,4.924680,landlocked
175,176,Baton Rouge,USA-LOU,Louisiana,US,United States,Provincial capital,220236,5,"100,000 to 249,999",...,2048113,,,,Inland perennial,US,0.026376,0.000021,3.256195,water
176,177,New Orleans,USA-LOU,Louisiana,US,United States,Other,390144,4,"250,000 to 499,999",...,2258909,Mississippi River,,,Inland perennial,US,8.947653,0.027984,4.058780,water
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,1527,Montpelier,USA-VRM,Vermont,US,United States,Provincial capital,7372,7,"Less than 50,000",...,2068522,,,,Inland perennial,US,0.071692,0.000076,7.755778,landlocked
1527,1528,Concord,USA-NHM,New Hampshire,US,United States,Provincial capital,43627,7,"Less than 50,000",...,2065854,,,,Inland perennial,US,0.165227,0.000097,2.895441,water
1528,1529,Boston,USA-MSS,Massachusetts,US,United States,Provincial capital,692600,3,"500,000 to 999,999",...,2257531,,,,Inland perennial,US,0.196315,0.000229,1.845970,water
1529,1530,Providence,USA-RHI,Rhode Island,US,United States,Provincial capital,179883,5,"100,000 to 249,999",...,2377205,Atlantic Ocean,,,Ocean or Sea,,2.583063,0.020320,2.284787,water


In [60]:
df[df.city_type=="landlocked"]

,FID,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,...,index__water,Name1,Name2,Name3,TYPE,ISO_CC,SHAPE_Length,SHAPE_Area,distance,city_type
10,11,Cuzco,PER-CUS,Cusco,PE,Peru,Provincial capital,428450,4,"250,000 to 499,999",...,2220440,Laguna Qoricocha,,,Inland perennial,PE,0.041373,0.000093,11.113561,landlocked
30,31,Talca,CHL-MAU,Maule,CI,Chile,Provincial capital,236724,5,"100,000 to 249,999",...,1198243,,,,Inland perennial,CL,0.017241,0.000013,18.309089,landlocked
32,33,Temuco,CHL-ARC,Araucania,CI,Chile,Provincial capital,302931,4,"250,000 to 499,999",...,1198152,,,,Inland perennial,CL,0.019454,0.000011,31.799283,landlocked
36,37,Copiapo,CHL-ATC,Atacama,CI,Chile,Provincial capital,171766,5,"100,000 to 249,999",...,2782678,Pacific Ocean,,,Ocean or Sea,,0.677627,0.019594,63.779497,landlocked
49,50,Rio Branco,BRA-ACR,Acre,BR,Brazil,Provincial capital,308545,4,"250,000 to 499,999",...,60774,,,,Inland perennial,BR,0.040504,0.000031,13.535620,landlocked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,2392,Masaka,UGA-STB,South Buganda,UG,Uganda,Provincial capital,103293,5,"100,000 to 249,999",...,2044012,,,,Inland perennial,UG,0.040123,0.000098,14.937287,landlocked
2411,2412,Mbarara,UGA-STH,Southern,UG,Uganda,Provincial capital,195160,5,"100,000 to 249,999",...,2044005,,,,Inland perennial,UG,0.026797,0.000030,17.365644,landlocked
2457,2458,Mbabane,SWZ-HHH,Hhohho,WZ,Eswatini,National capital,0,0,,...,2034069,,,,Inland perennial,SZ,0.107068,0.000147,13.616735,landlocked
2499,2500,Noril`sk,RUS-TAY,Taymyrskiy(Dolgano-Nenetskiy) avt.okrug,RS,Russia,Other,178363,5,"100,000 to 249,999",...,1645490,,,,Inland perennial,RU,0.064961,0.000055,13.785322,landlocked


In [68]:
result_rows = []

for country_code, group in df.groupby('FIPS_CNTRY'):
    # Find the capital city in the group
    capital_cities = group[group['capital']==1]
    
    if capital_cities.empty:
        continue  # No capital city in this country

    # Assuming there is only one capital city per country
    capital_city = capital_cities.iloc[0]

    capital_type = capital_city['city_type']

    # Determine the type of city to find
    if capital_type == 'landlocked':
    # Need to find a non-landlocked city
        candidate_cities = group[group['city_type'] != 'landlocked']
    else: # water
    # Need to find a landlocked city
        candidate_cities = group[group['city_type'] == 'landlocked']
       
    # Exclude the capital city from candidate cities
    candidate_cities = candidate_cities[candidate_cities['CITY_NAME'] != capital_city['CITY_NAME']]

    if candidate_cities.empty:
        # No suitable second city, just keep the capital city
        result_rows.append(capital_city)
    else:
        # Find the city with the closest population to the capital
        candidate_cities = candidate_cities.copy()
        candidate_cities['pop_diff'] = (candidate_cities['POP'] - capital_city['POP']).abs()

        # Get the city with the smallest population difference
        second_city = candidate_cities.loc[candidate_cities['pop_diff'].idxmin()]

        # Append both capital city and second city
        result_rows.extend([capital_city, second_city])

# Create the result dataframe
result = gpd.GeoDataFrame(result_rows, crs=projected_crs)

In [69]:
result

,FID,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,...,Name1,Name2,Name3,TYPE,ISO_CC,SHAPE_Length,SHAPE_Area,distance,city_type,pop_diff
295,296,Oranjestad,ABW,Aruba,AA,Aruba,National and provincial capital,28295,7,"Less than 50,000",...,Caribbean Sea,Atlantic Ocean,,Ocean or Sea,,1.283472,0.053652,1.449000,water,NaN
1555,1556,Saint John's,ATG,Antigua & Barbuda,AC,Antigua & Barbuda,National capital,0,0,,...,Caribbean Sea,Atlantic Ocean,,Ocean or Sea,,1.896968,0.045993,0.684450,water,NaN
2256,2257,Abu Dhabi,ARE-AZA,Abu Zaby,AE,United Arab Emirates,National and provincial capital,1539000,2,"1,000,000 to 4,999,999",...,Persian Gulf,Indian Ocean,,Ocean or Sea,,3.565310,0.047519,2.347458,water,NaN
1071,1072,Kabul,AFG-KAB,Kabol,AF,Afghanistan,National and provincial capital,4136000,2,"1,000,000 to 4,999,999",...,,,,Inland intermittent,AF,0.068363,0.000143,6.996153,water,NaN
1909,1910,Algiers,DZA-ALG,Alger,AG,Algeria,National and provincial capital,3608000,2,"1,000,000 to 4,999,999",...,Mediterranean Sea,Atlantic Ocean,,Ocean or Sea,,1.359198,0.047538,1.553963,water,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213,2214,Aden,YEM-ADN,Aden,YM,Yemen,Other,1247000,2,"1,000,000 to 4,999,999",...,Gulf of Aden,Indian Ocean,,Ocean or Sea,,1.106107,0.058359,0.129441,water,2573000.0
2381,2382,Lusaka,ZMB-LUS,Lusaka,ZA,Zambia,National and provincial capital,1747152,2,"1,000,000 to 4,999,999",...,,,,Inland perennial,ZM,0.015230,0.000016,14.661115,water,NaN
2374,2375,Kasama,ZMB-NRT,Northern,ZA,Zambia,Provincial capital,231824,5,"100,000 to 249,999",...,,,,Inland perennial,ZM,0.088829,0.000159,31.776473,landlocked,1515328.0
2311,2312,Harare,ZWE-MEA,Mashonaland East,ZI,Zimbabwe,National and provincial capital,1485231,2,"1,000,000 to 4,999,999",...,,,,Inland perennial,ZW,0.014086,0.000010,9.203922,water,NaN


In [72]:
result.to_csv("400cities_20241024.csv", index=False)